In [31]:
import pandas as pd 
import os 

In [32]:
table = pd.read_csv("./table/train_all_data_new.csv")
table

,date,title,author,url,image_link,hotNumber,is_hot,image_state
0,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/BdmZ7Ps.jpg,71,1,True
1,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/bBiw4IS.jpg,71,1,True
2,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/KDmRgeg.jpg,71,1,True
3,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/hRHpy8X.jpg,71,1,True
4,2023-Jan-1-00:26:06,[正妹] 周子瑜,ReiKuromiya,https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5...,https://i.imgur.com/P5rI8UM.jpg,71,1,True
...,...,...,...,...,...,...,...,...
67025,2023-Dec-31-21:57:42,[正妹] 潤娥,wafiea708,https://www.ptt.cc/bbs/Beauty/M.1704031064.A.D...,https://i.imgur.com/t0YhewP.gif,38,1,False
67026,2023-Dec-31-21:57:42,[正妹] 潤娥,wafiea708,https://www.ptt.cc/bbs/Beauty/M.1704031064.A.D...,https://i.imgur.com/B3SB2WY.jpg,38,1,True
67027,2023-Dec-31-21:57:42,[正妹] 潤娥,wafiea708,https://www.ptt.cc/bbs/Beauty/M.1704031064.A.D...,https://i.imgur.com/rCnbrGX.jpg,38,1,True
67028,2023-Dec-31-21:57:42,[正妹] 潤娥,wafiea708,https://www.ptt.cc/bbs/Beauty/M.1704031064.A.D...,https://i.imgur.com/zGIk6WD.jpg,38,1,True
